<a href="https://colab.research.google.com/github/jee365/ESAA/blob/main/%ED%9A%8C%EA%B7%802.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. 규제 선형 회귀 모델/ 회귀 트리**
릿지 회귀모델 및 회귀 트리를 이용하여 전복의 나이를 예측해 보세요.

전복의 ‘성별’, ‘키’, ‘지름’, ‘높이’, ‘전체무게’, ‘몸통무게’, ‘내장무게’, ‘껍질무게’를 입력하면 ‘껍질의 고리수’를 예측한 뒤 예측된 ‘껍질의 고리수’에 1.5를 더하면 전복의 나이가 됩니다.

In [3]:
# 기본 모듈 불러오기
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings(action='ignore')

### 1) 데이터 불러오기 및 데이터 선택

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
# 데이터 로드
data = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/abalone.csv")

# 성별 M은 Male, F는 Female, I는 Infant 이므로 따로 열 만들기
for label in "MFI":
    data[label] = data["Sex"] == label
data.drop('Sex', axis=1, inplace=True)

# X,y 데이터 선택
y = data.Rings.values
data.drop('Rings', axis=1, inplace=True)

X = data.values.astype(float)

## 2) train/test 분리
: 테스트 셋과 데이터 셋을 7:3 비율로 분리하세요.

In [6]:
# 필요한 모듈 불러오기
from sklearn.model_selection import train_test_split

In [7]:
# train과 test set 분리 (train:test = 7:3 비율로)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

## 3) 릿지 모델을 이용한 모델링
: 릿지 모델을 이용하여 전복의 나이를 예측해 보고 결과를 평가해 보세요.

 3-1) 릿지모델 생성, 교차검증, 평가에 필요한 모듈을 불러오세요.

In [8]:
#필요한 모듈 불러오기
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score

3-2) 릿지 모델을 생성하고 훈련하여 MSE와 RMSE를 구하세요. 릿지 모델 생성 시  alpha는 0.3으로 설정합니다. (교차 검증 하지 않습니다.)

In [10]:
#릿지모델 생성 및 훈련
ridge = Ridge(alpha=0.3)
ridge.fit(X_train, y_train)
#예측값 구하기
pred_ridge = ridge.predict(X_test)

#MSE와 RMSE구하기
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, pred_ridge)
rmse = np.sqrt(mse)
print(mse,rmse)


5.013158686100955 2.239008415817358


3-3) 교차검증을 이용하여 0, 0.1, 1, 10, 100 중  모델 성능이 가장 좋게 나오는 (rmse가 최소가 되는) alpha를 구하세요. 

In [11]:
# 주어진 alpha
alphas=[0, 0.1, 1, 10, 100]

In [14]:
#alphas list 값을 반복하면서 alpha에 따른 평균 rmse를 구함
#cross_val_score를 이용해 5 폴드의 평균 RMSE를 계산
for alpha in alphas:
  ridge = Ridge(alpha=alpha)
  neg_mse_scores = cross_val_score(ridge, X_train, y_train, scoring='neg_mean_squared_error',cv=5)
  avg_rmse = np.mean(np.sqrt(-1*neg_mse_scores))

  print('alpha {0} 일 때 5 폴드의 평균 RMSE : {1:.3f}'.format(alpha, avg_rmse))


alpha 0 일 때 5 폴드의 평균 RMSE : 2.192
alpha 0.1 일 때 5 폴드의 평균 RMSE : 2.189
alpha 1 일 때 5 폴드의 평균 RMSE : 2.192
alpha 10 일 때 5 폴드의 평균 RMSE : 2.336
alpha 100 일 때 5 폴드의 평균 RMSE : 2.604


## 4) XGBRegressor을 이용한 모델링
: XGBRegressor 모델을 이용하여 전복의 나이를 예측해 보고 결과를 평가해 보세요.

4-1) XGBRegressor 훈련, 평가에 필요한 모듈을 불러오세요.

In [15]:
# 필요한 모듈 임포트
from xgboost import XGBRegressor

4-2) XGBregressor 모델을 생성하고 훈련하여 MSE와 RMSE를 구하세요.

In [19]:
#모델 생성 및 훈련
xgb= XGBRegressor(n_estimators=1000)
xgb.fit(X_train, y_train)

#예측값 구하기
pred_xgb = xgb.predict(X_test)

#MSE와 RMSE 구하기
mse = mean_squared_error(y_test, pred_xgb)
rmse = np.sqrt(mse)

print(mse,rmse)

[09:55:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
5.337108949877299 2.3102183770971303


# **2. 로지스틱 회귀분석**
load_wine() 함수를 이용하여 와인 데이터를 다항분류하는 로지스틱 회귀모델을 생성하세요. 

In [20]:
# 필요한 모듈 불러오기
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

# 1) 데이터 불러오기 및 변수 선택

In [21]:
wine = load_wine()

wine_x = wine.data # x변수 
wine_y = wine.target # y변수

# 2) train/test 셋 분리

In [22]:
x_train, x_test, y_train, y_test = train_test_split(wine_x, wine_y, test_size=0.3, random_state=123)

x_train.shape 
y_train.shape 

(124,)

# 3) 로지스틱 회귀모델을 이용한 모델링

3-1) 로지스틱 회귀모델을 생성하여 훈련하세요.

In [23]:
from sklearn.linear_model import LogisticRegression
lr_clf=LogisticRegression()
lr_clf.fit(x_train,y_train)

LogisticRegression()

3-2) 회귀 모델의 기울기와 절편을 구하세요.

In [24]:
#기울기
print(lr_clf.coef_)

[[-1.99751167e-01  3.65450552e-01  1.49472112e-01 -1.88027149e-01
  -2.28495646e-02  1.95456485e-01  4.66701265e-01 -1.66796661e-02
   8.23897101e-02  6.75011473e-02 -1.92621856e-02  2.94915749e-01
   7.42524049e-03]
 [ 5.96475829e-01 -7.88672591e-01 -1.26181429e-01  2.12254654e-01
  -1.13504800e-02  3.15785912e-01  3.55147976e-01  3.34675580e-02
   2.79035669e-01 -1.17202474e+00  2.61165677e-01  4.64731544e-01
  -8.33633158e-03]
 [-3.96724662e-01  4.23222039e-01 -2.32906833e-02 -2.42275051e-02
   3.42000446e-02 -5.11242396e-01 -8.21849241e-01 -1.67878919e-02
  -3.61425379e-01  1.10452359e+00 -2.41903492e-01 -7.59647293e-01
   9.11091084e-04]]


In [25]:
#절편
print(lr_clf.intercept_)

[-0.04177837  0.09551046 -0.05373209]


3-3) 모델의 정확도를 구하세요.

In [30]:
#모델의 예측값 구하기
lr_preds = lr_clf.predict(x_test)

#모델의 정확도 구하기
from sklearn.metrics import accuracy_score
print('accuracy:',accuracy_score(y_test,lr_preds))

accuracy: 0.9629629629629629
